from Bartholomew & Kwakkel (2020):


2.2. Multi-scenario many-objective robust decision making
Multi-scenario MORDM (Watson and Kasprzyk, 2017) is a further
refinement of MORDM. The main contribution of MORDM was the
use of a MOEA for finding a set of promising candidate solutions
which together capture the key trade-offs amongst competing objectives. However, this search uses a single reference scenario, and it is
unlikely that solutions that are optimal in a given scenario are also
optimally robust. Multi-scenario MORDM (Fig. 1(c)) addresses this by
performing a search for candidate strategies for several different reference scenarios. The additional scenarios for which search is performed
are selected from regions in the deep uncertainty space where candidate
solutions found for the first reference scenario are failing to meet their
objectives. So, one performs the four MORDM steps, and based on
the insights from scenario discovery, additional scenarios are selected
for which search is also performed. The goal of this is to build a
more diverse set of policy alternatives which are Pareto optimal under
different scenarios.
The selection of scenarios after the first MORDM iteration is a critical step in multi-scenario MORDM (Eker and Kwakkel, 2018). Watson
and Kasprzyk (2017) suggest picking scenarios based on the scenario
discovery results. The number of scenarios to select is left to the
analyst. Clearly, if the number of scenarios for which a search is
conducted increases, the chance of finding solutions that are robust
during the re-evaluation also increases. However, this comes at a substantial computational cost. To assist in balancing comprehensiveness
and computational cost, while making scenario selection transparent
and reproducible, Eker and Kwakkel (2018) present an approach for
finding the most policy relevant and maximally diverse set of scenarios.
Policy relevance is defined as scenarios that lead to poor outcomes and
the diversity criterion is based on Carlsen et al. (2016).

# A. Multi-Scenario Many-Objective Robust Decision Making

### A.1. Problem formulation

### A.2. Scenario selection

### A.3. Generating candidate solutions

### A.4. Tradeoff analysis

**Signal-to-Noise**\
To quantify the robustness we can use multiple robustness metrices. The first one that we use is the **signal-to-noise ratio**. This is the mean of a dataset divided by the standard deviation. If we want to minimize the outcomes, a low mean and a low standard deviation is preferred.

In [ ]:
def signal_to_noise(data, direction):
    "Calculate the signal-to-noise ratio of a dataset with outcome directions (minimize or maximize)"
    mean = np.mean(data)
    std = np.std(data)
    
    if direction==ScalarOutcome.MAXIMIZE:
        return mean/std
    else:
        return mean*std

In [ ]:
# Empty dictionary to collect the ratio scores of every policy:
overall_scores = {}

# Loop over all policies and outcomes respectively
for policy in np.unique(experiments['policy']):
    scores = {}
    
    logical = experiments['policy']==policy
    
    for outcome in model.outcomes:
        value  = outcomes[outcome.name][logical]
        sn_ratio = s_to_n(value, outcome.kind)
        scores[outcome.name] = sn_ratio
    overall_scores[policy] = scores
    
# Create dataframe for visualisation purposes:
scores = pd.DataFrame.from_dict(overall_scores).T
scores

In [ ]:
# Visualise tradeoffs on a parallel plot:
from ema_workbench.analysis import parcoords

data = scores
limits = parcoords.get_limits(data)
limits.loc[0, ['A.1 Total Costs', 'A.1_Expected Number of Deaths', 'A.2 Total Costs', 'A.2_Expected Number of Deaths', 'A.3 Total Costs', 'A.3_Expected Number of Deaths','RfR Total Costs', 'Expected Evacuation Costs']] = 0

paraxes = parcoords.ParallelAxes(limits)
paraxes.plot(data)
paraxes.invert_axis('A.1 Total Costs')
plt.show()

**Minimax regret**
Another robustness metric that we use for our analysis is the **minimax regret** metric. This metric computes the regret for each option, then takes the ones with the maximum regret (worst-case), and then chooses the options that minimize this maximum regret. We define regret as the difference between the performance of a policy in a scenario and the performance of the best possible result in that scenario or the reference policy. The province of Gelderland is in favour of policy options with low maximum regret values, because the province is responsible for the safety of her region and pays the policies with government money. So, the province has a high level of risk aversion and therefore this robustness metric is suitable for our analysis.

Source: https://agupubs.onlinelibrary.wiley.com/doi/epdf/10.1002/2017EF000649